# Geometric Brownian Motion
https://en.wikipedia.org/wiki/Geometric_Brownian_motion  
A stochastic Process $S_t$ is a GBM if it follows the follow SDE:
$$dS_t=\mu S_t dt +\sigma S_t dW_t$$
where $W_t$ is a Wiener Process, $\mu$ (% drift) and $\sigma$(volatility)
By Ito's, we can solve the above eqution to
$$S_t = S_0 \exp((\mu - \frac{\sigma^2}{2})t+\sigma W_t)$$

## Wiener Process
https://en.wikipedia.org/wiki/Wiener_process  
There are four conditions for it to be wiener, but for this instance the property we are interested in is:
$$W_t \sim \mathcal{N}(0 ,t) = \sqrt{t} \mathcal{N}(0 ,1)$$

### kdb Formulae for GBM
in .stat namespace. In this case we are calculating $S_t/S_0$, the four arguements are
* $s = \sigma$
* $r = \mu$
* $t = t$
* $z = \mathcal{N}(0 ,1)$ (or a vectors of normal)|

In [1]:
gbm:{[s;r;t;z]exp (t*r-.5*s*s)+z*s*sqrt t} //multiply by 0.5 instead of %2, better

In [2]:
system "l qtips/stat.q"
//importing norminv
.stat.norminv //check whether function exist

{
 i:x<.5;
 x:?[i;1f-x;x];
 x:?[x<.92;cnorminv x;tnorminv x];
 x:?[i;neg x;x];
 x}


In [3]:
// generate path for 1 year of data, 252 biz days
.stat.gbm[.3;.05;1%252] .stat.norminv 252?1f //sigma = 30%, mu = 5%, dt=1 //sigma and mu for whole period? Y

0.9948897 1.00083 1.000777 0.9955672 0.9827334 0.9902536 1.015048 1.001668 1...


In [4]:
//above is daily returns 
prd .stat.gbm[.3;.05;1%252] .stat.norminv 252?1f //cummulative returns aggregating function: reduce length
prds .stat.gbm[.3;.05;1%252] .stat.norminv 252?1f //uniform function : return list of equal length

1.319982


0.9878928 1.016377 1.021751 1.019821 1.025445 1.069637 1.10095 1.108199 1.132..


In [30]:
avg 5000{prd .stat.gbm[.1;.05;1%500] .stat.norminv 500?1f}\1 //investigate on whether really 5%.

1.050408


## Path Generation with weekends consideration
note we use 2 arguement deltas, else first element will be itself(2001.01.01 === 365), with the first arguement, first element is 0

In [3]:
system "l qtips/util.q"
s:.3;r:.05;dt:.util.wday 2001.01.01+til 365;tm:deltas[first dt;dt]%365; 
prds .stat.gbm[s;r;tm] .stat.norminv (count dt)?1f

1 1.000687 1.00133 0.997638 0.9833011 0.9695503 0.9816554 0.9830126 0.9916588..


In [3]:
count tm //261 days including PH, maybe 252 is "better"

261


In [1]:
path:{[s;r;t]
    z:.stat.norminv count[t]?1f;
    prds .stat.gbm[s;r;deltas[first t;t]] z}

In [6]:
path[.3;.05](1%365.25)*.util.wday 2001.01.01+til 365

1 0.9872489 0.9881535 0.9815315 0.9782276 1.030696 1.03243 1.037051 1.030438 ..


In [7]:
.util.rng[0D00:01;0D09;0D15]%365D06 //365D06 = 1 year

0.001026694 0.001028595 0.001030497 0.001032398 0.001034299 0.0010362 0.00103..


In [9]:
path[.3;.05] .util.rng[0D00:01;0D09;0D15]%365D06 //path for 1 day //minute data

1 0.9994987 0.9989085 0.9998023 0.9995217 1.000419 0.9997889 1.000427 1.00059..


In [10]:
.util.rnd[0.05] 100* path[.3;.05] .util.rng[0D00:01;0D09;0D15]%365D06 //path for 1 day //imagine S_0 = 100

100 100.05 100.05 100 100 100 100 100 100 100 99.95 100 100 100 100.05 100 10..
